# Model Training

### 1.1. Import Data and Required Packages
Importing Pandas, Numpy, Matplotlib, Seaborn and Warnings Libraries

In [81]:
# basic import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# modeling import
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

# warning import
import warnings

### Import the CSV Data as Pandas DataFrame

In [82]:
df = pd.read_csv('data/student_performance.csv')

# show top 5 records
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Preparing X and Y variables

In [83]:
X = df.drop(columns=['math_score'], axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [84]:
print('Categories in "gener" variable: ', df['gender'].unique())

print('Categories in "race_ethnicity" variable: ', df['race_ethnicity'].unique())

print('Categories in "parental level of education" variable: ', df['parental_level_of_education'].unique())

print('Categories in "lunch" variable: ', df['lunch'].unique())

print('Categories in "test preparation course" variable: ', df['test_preparation_course'].unique())

Categories in "gener" variable:  ['female' 'male']
Categories in "race_ethnicity" variable:  ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in "parental level of education" variable:  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in "lunch" variable:  ['standard' 'free/reduced']
Categories in "test preparation course" variable:  ['none' 'completed']


In [85]:
y = df['math_score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [86]:
# create column transform with 3 types of transformers
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', oh_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_features)
    ]
)

In [87]:
X = preprocessor.fit_transform(X)
X.shape

(1000, 19)

In [88]:
# separate dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

### Making an Evaluate Function to give all metrics after training model

In [89]:
def metrics(ground_truth, predicted):
    """Calculate all metric after finising model training

    Args:
        ground_truth (float): true value
        predicted (float): predicted value
    """
    mae = mean_absolute_error(ground_truth, predicted)
    mse = mean_squared_error(ground_truth, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(ground_truth, predicted)

    return mae, mse, rmse, r2_square

In [90]:
models = {
    'Linear Regression': LinearRegression(),
    'Lasso': Lasso(),
    'Ridge': Ridge(),
    'k-Neibors Regressor': KNeighborsRegressor(),
    'Decision Tree': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'XGBRegressor': XGBRegressor(),
    'CatBoosting Regressor': CatBoostRegressor(verbose=False),
    'AdaBoost Regressor': AdaBoostRegressor()
}

In [91]:
model_list = []
r2_list = []

for i in range(len(list(models))):
    model = list(models.values())[i]
    # train model
    model.fit(X_train, y_train)
    # predict
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # evaluate model on train and test sets
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2_score = metrics(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2_score = metrics(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for training set: ')
    print('Mean Squared Error: {:.4f}'.format(model_train_mse))
    print('Root Mean Squared Error: {:.4f}'.format(model_train_rmse))
    print('Mean Absoluted Error: {:.4f}'.format(model_train_mae))
    print('R2 score: {:.4f}'.format(model_train_r2_score))

    print('---'*20)

    print('Model performance for test set: ')
    print('Mean Squared Error: {:.4f}'.format(model_test_mse))
    print('Root Mean Squared Error: {:.4f}'.format(model_test_rmse))
    print('Mean Absoluted Error: {:.4f}'.format(model_test_mae))
    print('R2 score: {:.4f}'.format(model_test_r2_score))

    r2_list.append(model_test_r2_score)

    print('='*35)
    print('\n')

Linear Regression
Model performance for training set: 
Mean Squared Error: 28.3476
Root Mean Squared Error: 5.3242
Mean Absoluted Error: 4.2673
R2 score: 0.8743
------------------------------------------------------------
Model performance for test set: 
Mean Squared Error: 29.1108
Root Mean Squared Error: 5.3954
Mean Absoluted Error: 4.2148
R2 score: 0.8804


Lasso
Model performance for training set: 
Mean Squared Error: 43.4784
Root Mean Squared Error: 6.5938
Mean Absoluted Error: 5.2063
R2 score: 0.8071
------------------------------------------------------------
Model performance for test set: 
Mean Squared Error: 42.5064
Root Mean Squared Error: 6.5197
Mean Absoluted Error: 5.1579
R2 score: 0.8253


Ridge
Model performance for training set: 
Mean Squared Error: 28.3378
Root Mean Squared Error: 5.3233
Mean Absoluted Error: 4.2650
R2 score: 0.8743
------------------------------------------------------------
Model performance for test set: 
Mean Squared Error: 29.0563
Root Mean Squar

### Results table

In [92]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=['R2_Score'], ascending=False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.880369
7,CatBoosting Regressor,0.851632
5,Random Forest Regressor,0.849708
8,AdaBoost Regressor,0.844269
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,k-Neibors Regressor,0.783813
4,Decision Tree,0.738759


### Linear Regression

In [93]:
linear_model = LinearRegression(fit_intercept=True)
# train model
linear_model = linear_model.fit(X_train, y_train)

# predict
y_test_pred = linear_model.predict(X_test)

r2_score = r2_score(y_test, y_test_pred) * 100
print('Accuracy of the model is {:.4f}'.format(r2_score))

Accuracy of the model is 88.0369


### Plot y_test_pred and y_test

In [ ]:
plt.scatter(y_test, y_test_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')